Просто расстановка ударений (ruaccent)  

In [ ]:
from ruaccent import RUAccent
 
accentizer = RUAccent()
accentizer.load(omograph_model_size='big_poetry', use_dictionary=True, tiny_mode=False, device="CUDA")

text = """ вахтёр - вахтер
вёдро - ведро
всё - все
вспольё - всполье
ёрник - ерник
желёзка - железка
залёжный - залежный
замётка - заметка
засолённый - засоленный
заторможённый - заторможенный
крёстный - крестный
лён - лен
лётом - летом
маркёр - маркер
маркёрный - маркерный
мётка - метка
нёбо - небо
падёж - падеж
падёжный - падежный
перёд - перед
письмённый - письменный
примётка - приметка
совершённое - совершенное
солитёр - солитер
стартёр - стартер
стартёрный - стартерный
стрёмить - стремить
твёрдо - твердо
фён - фен
шабёр - шабер
"""
print(accentizer.process_all(text)) 


ИНФО
__init__ ()
_process_accent (text, stress_usages)
_process_omographs (text)
_process_yo (words, sentence)
count_vowels (text)
delete_spaces_before_punc (text)
extract_entities (data)
has_punctuation (text)
load (omograph_model_size='turbo2', use_dictionary=False, custom_dict={}, custom_homographs={}, device='CPU', repo='ruaccent/accentuator', workdir=None, tiny_mode=False)
process_all (text, skip_regex=None)
process_all_internal (text)
process_yo (text)

In [ ]:
from ruaccent import RUAccent
import inspect  
import sys
import os
sys.path.append(os.path.abspath(".."))  # добавить родительскую папку
from DICT_ACC_MY import DICT_ACC_MY
 
accentizer = RUAccent()
accentizer.load(omograph_model_size='big_poetry', use_dictionary=True, tiny_mode=False, device="CUDA")
# accentizer.load (omograph_model_size='big_poetry', use_dictionary=True, custom_dict=DICT_ACC_MY, custom_homographs={}, device='CPU', tiny_mode=False)

text = """ карасем все вместе идут домой то что  карасем зовется  жопа """
print(accentizer.process_all(text)) 
print(accentizer.count_vowels("белок"))
print(accentizer.delete_spaces_before_punc("белок , торты ."))
print(accentizer.has_punctuation("  всё то что ? зовётся  жопа " ))
print(accentizer.process_all_internal(text)) 
print(accentizer.process_yo(text)) 


# методы класса RUAccent ??
for name, func in inspect.getmembers(accentizer, predicate=inspect.ismethod):
    print(name, inspect.signature(func)) 


Сведения о внутренних функциях ruaccent   !!!  

In [ ]:
import inspect
print(inspect.getsource(RUAccent.__init__))
print(inspect.getsource(RUAccent._process_accent))
print(inspect.getsource(RUAccent._process_omographs))
print(inspect.getsource(RUAccent._process_yo))
print(inspect.getsource(RUAccent.count_vowels))
print(inspect.getsource(RUAccent.delete_spaces_before_punc))
print(inspect.getsource(RUAccent.extract_entities))
print(inspect.getsource(RUAccent.has_punctuation))
print(inspect.getsource(RUAccent.load))
print(inspect.getsource(RUAccent.process_all))
print(inspect.getsource(RUAccent.process_all_internal))
print(inspect.getsource(RUAccent.process_yo))



ПОДРОБНЕЕ о ruaccent

In [ ]:
from ruaccent import RUAccent

accentizer = RUAccent()
accentizer.load(
    omograph_model_size="turbo3.1",  # или tiny/turbo/big_poetry
    use_dictionary=True,
    custom_dict={'замок': 'зам+ок'},  # пример: переопределить ударение (не работает?)
    device="CUDA",                  # или "CPU"
    workdir=None,                   # куда сохранять модели
    tiny_mode=False                 # упрощённый режим без словаря
)

text = "Она пришла домой и увидела замок на двери."
accented = accentizer.process_all(text)
print(accented)


Он+а пришл+а дом+ой и ув+идела з+амок на двер+и.


In [2]:
import ruaccent, os
print(os.listdir(ruaccent.__path__[0]))


['.cache', 'accent_model.py', 'char_tokenizer.py', 'dictionary', 'koziev', 'nn', 'omograph_model.py', 'ruaccent.py', 'rule_accent_engine.py', 'stress_usage_model.py', 'text_postprocessor.py', 'text_preprocessor.py', 'text_split.py', 'yo_homograph_model.py', '__init__.py', '__pycache__']


МНОГО ИНФЫ  о ruaccent  (+ onnxruntime)

In [1]:
import os
import re
import ruaccent
from ruaccent import RUAccent

print("ruaccent version:", getattr(ruaccent, "__version__", "unknown"))

# --- helper: сохранить case и заменить слова на размеченные ---


def annotate_custom(text: str, custom: dict[str, str]) -> str:
    if not custom:
        return text
    # строим паттерн по ключам (lowercase)
    keys = sorted(custom.keys(), key=len, reverse=True)
    pat = re.compile(r'\b(' + '|'.join(re.escape(k) for k in keys) + r')\b',
                     flags=re.IGNORECASE | re.UNICODE)

    def repl(m):
        orig = m.group(0)
        key = orig.lower()
        ann = custom.get(key)
        if not ann:
            return orig
        # сохранить капитализацию первой буквы
        if orig and orig[0].isupper():
            return ann.capitalize()
        return ann

    return pat.sub(repl, text)


# --- load accentizer (CPU first, чтобы исключить GPU-ошибки) ---
accentizer = RUAccent()
accentizer.load(
    omograph_model_size="turbo3.1",
    use_dictionary=True,
    custom_dict={'замок': 'зам+ок'},  # пробуем явно
    device="CPU",
    tiny_mode=False
)

# --- быстрый debug: посмотреть атрибуты акцентайзера и путь к модели ---
print("\n--- Accentizer attributes (select) ---")
for k, v in sorted(accentizer.__dict__.items()):
    if any(sub in k.lower() for sub in ("dict", "omograph", "workdir", "model", "path")):
        print(k, ":", type(v))

# попытка найти путь до onnx модели
omap = getattr(accentizer, "omograph_models_paths", None)
wd = getattr(accentizer, "workdir", None)
print("\nworkdir:", wd)
print("omograph_models_paths:", omap)

if wd and omap and "turbo3.1" in omap:
    rel = omap["turbo3.1"]
    candidate = os.path.join(wd, rel.lstrip("/\\"))
    print("expected model path:", candidate,
          "exists:", os.path.exists(candidate))

# onnxruntime провайдеры (показывает, доступен ли CUDAExecutionProvider)
try:
    import onnxruntime as ort
    print("\nonnxruntime available providers:", ort.get_available_providers())
    sess = getattr(getattr(accentizer, "omograph_model",
                   object()), "session", None)
    if sess:
        try:
            print("accentizer session providers:", sess.get_providers())
        except Exception as e:
            print("can't read session providers:", e)
except Exception as e:
    print("\nonnxruntime import failed or not installed:", e)

# --- проверка custom dict в действии ---
text = "Она пришла домой и увидела замок на двери."
print("\nOriginal text:", text)
print("accentizer.process_all (без предобработки):")
print("  ", accentizer.process_all(text))

text_annot = annotate_custom(text, {'замок': 'зам+ок'})
print("\nAfter annotate_custom (подставили 'зам+ок'):")
print("  ", text_annot)
print("accentizer.process_all (после подстановки):")
print("  ", accentizer.process_all(text_annot))

ruaccent version: 1.5.8.3

--- Accentizer attributes (select) ---
accent_model : <class 'ruaccent.accent_model.AccentModel'>
accentuator_paths : <class 'list'>
custom_dict : <class 'dict'>
koziev_paths : <class 'list'>
module_path : <class 'str'>
omograph_model : <class 'ruaccent.omograph_model.OmographModel'>
omograph_models_paths : <class 'dict'>
omographs : <class 'dict'>
stress_usage_predictor : <class 'ruaccent.stress_usage_model.StressUsagePredictorModel'>
workdir : <class 'str'>
yo_homograph_model : <class 'ruaccent.yo_homograph_model.YoHomographModel'>
yo_homographs : <class 'dict'>

workdir: C:\Users\Odins\new-git\2_1_Prog\.venv\Lib\site-packages\ruaccent
omograph_models_paths: {'big_poetry': '/nn/nn_omograph/big_poetry', 'medium_poetry': '/nn/nn_omograph/medium_poetry', 'small_poetry': '/nn/nn_omograph/small_poetry', 'turbo': '/nn/nn_omograph/turbo', 'turbo2': '/nn/nn_omograph/turbo2', 'turbo3': '/nn/nn_omograph/turbo3', 'turbo3.1': '/nn/nn_omograph/turbo3.1', 'tiny': '/nn/nn

проверка откуда пакет и его версия

In [ ]:
import sys
import ruaccent

print(ruaccent.__file__)
print(sys.executable)

print(ruaccent.__version__)


инфо коротко

In [ ]:
import torch
import onnxruntime as ort
print("torch.cuda.is_available():", torch.cuda.is_available())
print("torch.version.cuda:", torch.version.cuda)
print("CUDA device:", torch.cuda.get_device_name(
    0) if torch.cuda.is_available() else "none")
print("onnxruntime providers:", ort.get_available_providers())

=============================================

другой способ сравнения ритмов 
(нужна коррекция - ударения однослоговых   и  "ё")

In [ ]:
import re
from ruaccent import RUAccent

# -----------------------------
# 1. Настройка RUAccent
# -----------------------------
accentizer = RUAccent()
accentizer.load(omograph_model_size="turbo3.1",
                use_dictionary=True, tiny_mode=False)

# -----------------------------
# 2. Константы
# -----------------------------
VOWELS = "аеёиоуыэюяАЕЁИОУЫЭЮЯ"

# -----------------------------
# 3. Анализ ритма по слогам
# -----------------------------


def analyze_rhythm(accented_line: str) -> str:
    """
    Получает строку с ударениями (ruaccent, + перед ударной гласной),
    возвращает схему 0/1 по слогам.
    """
    rhythm = []
    i = 0
    while i < len(accented_line):
        ch = accented_line[i]
        if ch == "+" and i + 1 < len(accented_line) and accented_line[i+1] in VOWELS:
            rhythm.append("1")   # ударная гласная
            i += 1               # пропускаем саму гласную
        elif ch in VOWELS:
            rhythm.append("0")   # безударная гласная
        i += 1
    return "".join(rhythm)

# -----------------------------
# 4. Проверка схемы
# -----------------------------


def check_rhythm_scheme(rhythm: str, scheme: str) -> bool:
    return rhythm == scheme

# -----------------------------
# 5. Полный анализ стихотворения
# -----------------------------


def _analyze_poem(poem: str, scheme: str) -> list[tuple[str, str, str, bool]]:
    """
    Анализирует стихотворение.
    Возвращает список кортежей:
    (оригинал строки, строка с ударениями, ритм, совпадает ли со схемой).
    """
    results = []
    for line in poem.splitlines():
        clean_line = line.strip()
        if not clean_line:
            continue
        accented_line = accentizer.process_all(clean_line)
        rhythm = analyze_rhythm(accented_line)
        matches = check_rhythm_scheme(rhythm, scheme)
        results.append((clean_line, accented_line, rhythm, matches))
    return results


# -----------------------------
# 6. Пример использования
# -----------------------------
SCHEME = "0101"  # для примера

poem = """привет салют 
на перекрёстке двух дорог
я ждал вас там четыре года
и вот вы здесь"""

results = _analyze_poem(poem, SCHEME)

for line, accented, rhythm, matches in results:
    print(f"Строка: {line}")
    print(f"Ударения: {accented}")
    print(f"Ритм: {rhythm}")
    print(f"Соответствует схеме: {matches}\n")

=====================================================

дб     УДАРЕНИЯ (ОБХОД ruaccent) 

In [ ]:
import regex
from ruaccent import RUAccent


def try_annotate_custom(accentizer, text, custom_dict):
    """Попытаться вызвать annotate_custom в известных вариантах сигнатуры."""
    f = getattr(accentizer, "annotate_custom", None)
    if f is None:
        return None
    # пробуем разные варианты вызова — сигнатура у разных версий может отличаться
    try:
        return f(text, custom_dict)
    except TypeError:
        try:
            return f(custom_dict)
        except TypeError:
            try:
                return f(text)
            except Exception:
                return None
    except Exception:
        return None


def apply_custom_postprocess(processed_text: str, custom_dict: dict) -> str:
    """
    В processed_text (строка, выход accentizer.process_all) находит слова,
    соответствующие ключам custom_dict (игнорируя '+' и регистр), и
    заменяет их на значение из custom_dict. Сохраняет первую букву в верхнем регистре,
    если в исходном токене была заглавная.
    custom_dict: {'замок': 'зам+ок', ...}
    """
    # нормализуем ключи: удалить не-буквы, привести к lower
    norm_map = {regex.sub(r'[^\p{L}]', '', k).lower(): v for k, v in custom_dict.items()}

    # разобьём processed_text на токены: слова (буквы и +) и всё остальное
    tokens = regex.findall(r'(?:\+?[\p{L}\+]+|[^\p{L}\+]+)', processed_text)

    out = []
    for tok in tokens:
        if regex.fullmatch(r'[\p{L}\+]+', tok):
            # слово-подобный токен: убираем '+' и нормализуем
            stripped = tok.replace('+', '')
            norm = regex.sub(r'[^\p{L}]', '', stripped).lower()
            if norm in norm_map:
                replacement = norm_map[norm]
                # сохранить капитализацию первой буквы, если в исходном токене первая буква была заглавной
                first_orig_letter = next(
                    (ch for ch in stripped if ch.isalpha()), None)
                if first_orig_letter and first_orig_letter.isupper():
                    # найти индекс первой буквы в replacement и сделать её заглавной
                    rep_list = list(replacement)
                    for i, ch in enumerate(rep_list):
                        if ch.isalpha():
                            rep_list[i] = rep_list[i].upper()
                            break
                    replacement = ''.join(rep_list)
                out.append(replacement)
                continue
        out.append(tok)
    return ''.join(out)


def accent_with_custom(accentizer: RUAccent, text: str, custom_dict: dict, try_annotate=True) -> str:
    """
    Главная обёртка: пытается annotate_custom, запускает process_all,
    затем принудительно применяет custom_dict постфактум.
    """
    # обновим accentizer.custom_dict (если оно есть) — иногда помогает
    try:
        cd = getattr(accentizer, "custom_dict", None)
        if isinstance(cd, dict):
            cd.update(custom_dict)
    except Exception:
        pass

    annotated_text = None
    if try_annotate:
        annotated_text = try_annotate_custom(accentizer, text, custom_dict)

    if annotated_text is not None:
        processed = accentizer.process_all(annotated_text)
    else:
        processed = accentizer.process_all(text)

    final = apply_custom_postprocess(processed, custom_dict)
    return final


# --------------------- пример использования ---------------------
if __name__ == "__main__":
    accentizer = RUAccent()
    accentizer.load(
        omograph_model_size="turbo3.1",
        use_dictionary=True,
        custom_dict={},   # можно передать сразу, но wrapper обновит тоже   НЕ РАБОТАЕТ ???
        device="CUDA",     # сначала тестируем на CPU; чтобы использовать CUDA — см. выше
        tiny_mode=False, 
        # workdir=None   где хранятся модели (по умолчанию '~/.ruaccent'
    )

    text = "Она пришла домой и увидела замок на двери."
    custom = {"замок": "зам+ок"}   # ключи — в простой форме, значения — с '+'
    out = accent_with_custom(accentizer, text, custom)
    print(out)
    print(accentizer.process_all(text))  # для сравнения

дб   ЭТО В _accentize.py   (ещё см GPT)    ОБЁРТКИ НА ruaccent (?)

In [ ]:
import regex
from ruaccent import RUAccent

# какая то фигня - выкинул
def try_annotate_custom(accentizer, text, custom_dict):
    f = getattr(accentizer, "annotate_custom", None)
    if f is None:
        return None
    try:
        return f(text, custom_dict)
    except TypeError:
        try:
            return f(custom_dict)
        except TypeError:
            try:
                return f(text)
            except Exception:
                return None
    except Exception:
        return None

#  ? сделать   сохранить от изменения "е" и "ё" (или есть отключение ёфикатора?)
def apply_custom_postprocess(processed_text: str, custom_dict: dict) -> str:
    norm_map = {regex.sub(r'[^\p{L}]', '', k).lower(): v for k, v in custom_dict.items()}
    tokens = regex.findall(r'(?:\+?[\p{L}\+]+|[^\p{L}\+]+)', processed_text)

    out = []
    for tok in tokens:
        if regex.fullmatch(r'[\p{L}\+]+', tok):
            stripped = tok.replace('+', '')
            norm = regex.sub(r'[^\p{L}]', '', stripped).lower()
            if norm in norm_map:
                replacement = norm_map[norm]
                # сохранить заглавную первую букву
                first_orig_letter = next((ch for ch in stripped if ch.isalpha()), None)
                if first_orig_letter and first_orig_letter.isupper():
                    rep_list = list(replacement)
                    for i, ch in enumerate(rep_list):
                        if ch.isalpha():
                            rep_list[i] = rep_list[i].upper()
                            break
                    replacement = ''.join(rep_list)
                out.append(replacement)
                continue
        out.append(tok)
    return ''.join(out)


def accentize(
    text: str,
    accentizer: RUAccent = None,
    use_manual=True,          # CUSTOM_ACCENT
    custom_dict=None,         # CUSTOM_DICT
    try_annotate=True,        # пробовать annotate_custom (если есть у самого accentizer)
    bypass_ruaccent=False,    # полный обход ruaccent
    debug=False               # печатать шаги
) -> str:
    """
    Универсальная обёртка над ruaccent.
    text            — входной текст
    accentizer      — экземпляр RUAccent (или None → bypass)
    use_manual      — True: сохранять слова с '+'
    custom_dict     — словарь {'замок': 'зам+ок'}
    try_annotate    — пробовать accentizer.annotate_custom (если есть)
    bypass_ruaccent — True: игнорировать ruaccent, работать только словарём и '+'
    debug           — печатать выбранный режим
    """
    if custom_dict is None:
        custom_dict = {}

    # авто-bypass если accentizer не передан
    if accentizer is None:
        bypass_ruaccent = True
        if debug:
            print("[DEBUG] accentizer=None → bypass_ruaccent включён")

    # --- 1. Сохраняем ручные акценты ---
    manual_accents = []
    words = text.split()
    clean_text = text
    if use_manual:
        manual_accents = [(i, w) for i, w in enumerate(words) if "+" in w]
        clean_text = clean_text.replace("+", "")
        if debug and manual_accents:
            print(f"[DEBUG] Найдено {len(manual_accents)} слов с ручными акцентами")

    # --- 2. Основная логика ---
    if bypass_ruaccent:
        processed = clean_text
        if debug:
            print("[DEBUG] Пропускаем ruaccent, работаем только словарём/ручными акцентами")
    else:
        annotated_text = None
        if custom_dict and try_annotate:
            annotated_text = try_annotate_custom(accentizer, clean_text, custom_dict)
            if debug and annotated_text is not None:
                print("[DEBUG] Успешно вызван annotate_custom")

        if annotated_text is not None:
            processed = accentizer.process_all(annotated_text)
            if debug:
                print("[DEBUG] Запуск process_all на annotate_custom")
        else:
            processed = accentizer.process_all(clean_text)
            if debug:
                print("[DEBUG] Запуск process_all напрямую")

    # --- 3. Восстанавливаем ручные акценты ---
    if manual_accents:
        processed_words = processed.split()
        for idx, manual_word in manual_accents:
            if idx < len(processed_words):
                processed_words[idx] = manual_word
        processed = " ".join(processed_words)
        if debug:
            print("[DEBUG] Ручные акценты восстановлены")

    # --- 4. Применяем custom_dict ---
    if custom_dict:
        processed = apply_custom_postprocess(processed, custom_dict)
        if debug:
            print(f"[DEBUG] Применён custom_dict (размер={len(custom_dict)})")

    return processed


# ---------------- пример ----------------
if __name__ == "__main__":
    accentizer = RUAccent()
    accentizer.load(
        omograph_model_size="turbo2",
        use_dictionary=True,
        device="CPU",
    )

    txt = "б+езрукий паслён кричит замок на двери"
    custom = {"замок": "зам+ок"}

    print("ruaccent + словарь:")
    print(accentize(txt, accentizer, use_manual=True, custom_dict=custom, debug=True))
    print("\nтолько словарь (bypass):")
    print(accentize(txt, custom_dict=custom, debug=True))  # accentizer=None → bypass
    print("\nтолько ruaccent:") 
    print(accentize(txt, accentizer, use_manual=False, debug=True))


дб   2.0

In [ ]:
import regex
from ruaccent import RUAccent


def try_annotate_custom(accentizer, text, custom_dict):
    """Универсальный вызов accentizer.annotate_custom (если он есть)."""
    f = getattr(accentizer, "annotate_custom", None)
    if f is None:
        return None
    try:
        return f(text, custom_dict)
    except TypeError:
        try:
            return f(custom_dict)
        except TypeError:
            try:
                return f(text)
            except Exception:
                return None
    except Exception:
        return None


def apply_custom_postprocess(processed_text: str, custom_dict: dict) -> str:
    """Постобработка словарём: учитывает регистр, чистит символы."""
    norm_map = {regex.sub(r"[^\p{L}]", "", k).lower(): v for k, v in custom_dict.items()}
    tokens = regex.findall(r"(?:\+?[\p{L}\+]+|[^\p{L}\+]+)", processed_text)

    out = []
    for tok in tokens:
        if regex.fullmatch(r"[\p{L}\+]+", tok):
            stripped = tok.replace("+", "")
            norm = regex.sub(r"[^\p{L}]", "", stripped).lower()
            if norm in norm_map:
                replacement = norm_map[norm]
                # сохраняем заглавную первую букву
                first_orig_letter = next((ch for ch in stripped if ch.isalpha()), None)
                if first_orig_letter and first_orig_letter.isupper():
                    rep_list = list(replacement)
                    for i, ch in enumerate(rep_list):
                        if ch.isalpha():
                            rep_list[i] = rep_list[i].upper()
                            break
                    replacement = "".join(rep_list)
                out.append(replacement)
                continue
        out.append(tok)
    return "".join(out)


def load_default_dict():
    """Простейший дефолтный словарь — заглушка."""
    return {"замок": "зам+ок", "атлас": "атл+ас"}


def accentize(
    text: str,
    accentizer: RUAccent = None,
    use_manual=True,          # сохранять слова с '+'
    custom_dict=None,         # None → дефолтный словарь, {} → словарь отключен
    try_annotate=True,        # пробовать accentizer.annotate_custom
    bypass_ruaccent=False,    # полный обход ruaccent
    debug=False,              # печатать шаги
) -> str:
    """
    Универсальная обёртка над ruaccent.
    """

    # --- 1. Разбираемся со словарём ---
    if custom_dict is None:
        custom_dict = load_default_dict()
        if debug:
            print("[DEBUG] custom_dict=None → загружен дефолтный словарь")
    elif custom_dict == {}:
        custom_dict = None
        if debug:
            print("[DEBUG] custom_dict={} → словарь отключён")

    # --- 2. Авто-инициализация accentizer ---
    if not bypass_ruaccent and accentizer is None:
        if debug:
            print("[DEBUG] accentizer=None → создаём новый RUAccent()")
        accentizer = RUAccent()
        accentizer.load(omograph_model_size="turbo2", use_dictionary=True, device="CPU")

    # --- 3. Сохраняем ручные акценты ---
    manual_accents = []
    words = text.split()
    clean_text = text
    if use_manual:
        manual_accents = [(i, w) for i, w in enumerate(words) if "+" in w]
        clean_text = clean_text.replace("+", "")
        if debug and manual_accents:
            print(f"[DEBUG] Найдено {len(manual_accents)} слов с ручными акцентами")

    # --- 4. Основная логика ---
    if bypass_ruaccent:
        processed = clean_text
        if debug:
            print("[DEBUG] bypass_ruaccent=True → работаем только словарём/ручными акцентами")
    else:
        annotated_text = None
        if custom_dict and try_annotate:
            annotated_text = try_annotate_custom(accentizer, clean_text, custom_dict)
            if debug and annotated_text is not None:
                print("[DEBUG] Успешно вызван annotate_custom")

        if annotated_text is not None:
            processed = accentizer.process_all(annotated_text)
            if debug:
                print("[DEBUG] Запуск process_all на annotate_custom")
        else:
            processed = accentizer.process_all(clean_text)
            if debug:
                print("[DEBUG] Запуск process_all напрямую")

    # --- 5. Восстанавливаем ручные акценты ---
    if manual_accents:
        processed_words = processed.split()
        for idx, manual_word in manual_accents:
            if idx < len(processed_words):
                processed_words[idx] = manual_word
        processed = " ".join(processed_words)
        if debug:
            print("[DEBUG] Ручные акценты восстановлены")

    # --- 6. Применяем custom_dict ---
    if custom_dict:
        processed = apply_custom_postprocess(processed, custom_dict)
        if debug:
            print(f"[DEBUG] Применён custom_dict (размер={len(custom_dict)})")

    return processed


# ---------------- пример ----------------
if __name__ == "__main__":
    txt = "б+езрукий паслён кричит замок на двери"

    print("ruaccent + словарь:")
    print(accentize(txt, use_manual=True, debug=True))

    print("\nтолько словарь (bypass):")
    print(accentize(txt, custom_dict={"замок": "зам+ок"}, bypass_ruaccent=True, debug=True))

    print("\nтолько ruaccent:")
    print(accentize(txt, custom_dict={}, use_manual=False, debug=True))


==============================================

дб  попытка проставить ударения в составных словах

In [ ]:
from ruaccent import RUAccent

accentizer = RUAccent()
accentizer.load(omograph_model_size='big_poetry',
                use_dictionary=True, tiny_mode=False) #  tiny_mode лучше иногда ?

def complex_words_postprocess(processed_text: str, accentizer: RUAccent) :
    """
    Обрабатывает слова, которые не получили ударений:
      1) пробуем «подрезку» слова от начала (cut-rule),
      2) если всё ещё нет — ставим '+' перед 'ё' (yo-rule).
    Возвращает (обновлённый текст, список изменений).
    """
    vowels = "аеёиоуыэюяАЕЁИОУЫЭЮЯ"
    words = processed_text.split()
    post_changes = []
    saved = []

    for i, w in enumerate(words):
        # пропускаем: односложные или уже с "+"
        if sum(ch in vowels for ch in w) <= 1 or "+" in w:
            continue

        # --- 1. cut-rule ---
        applied = False
        for j in range(min(7, max(0, len(w) - 3))): # ?? не оставлять коротких обрезков-слов и не брать много с начала
            cut_w = w[j:]
            acc_cut_w = accentizer.process_all(cut_w)
            if "+" in acc_cut_w:
                new_w = w[:j] + acc_cut_w.replace("+", "?")
                post_changes.append((i, w, new_w, "cut", acc_cut_w))
                saved.insert(0, new_w) 
                applied = True
                break

        if applied:
            continue

        # --- 2. ё-rule ---
        if "ё" in w:
            new_w = w.replace("ё", "?ё")     # отмечаем все ё
            post_changes.append((i, w, new_w, "yo"))
            saved.insert(0, new_w) 

        else: saved.append(w)     # слова оставшиеся без "+"

    return post_changes, saved


text = accentizer.process_all("""сёгун  трёхколёсный четырёхколёсный четырёхкамерный
вертолётостроение самолётостроение сёрфингиста """)
print(text)
print(complex_words_postprocess(text, accentizer)[0]) 
print(complex_words_postprocess(text, accentizer)[1]) 


====================================

(от ии. не проверен)   Делим текст на куски (чанки) для nn. Безопасный вызов accentizer.process_all.  (было для accentize.py)   
+ см   py библиотечка razdel (для токенизации русских слов)  ??

In [ ]:
#  (от ии. не проверен)   Делим текст на куски (чанки) для nn. Безопасный вызов accentizer.process_all.
def _safe_process(accentizer, text: str, max_len: int = 400):
    """
    Аккуратно обрабатывает текст, деля его на чанки:
    1. Собираем несколько абзацев вместе, если помещаются в max_len.
    2. Если абзац длинный — режем его по строкам.
    3. Если и строка длинная — делим по словам.
    """

    def chunk_text_by_lines(text: str, max_chars: int = 400) -> list[str]:
        """Делим текст по абзацам, объединяя их до max_chars, но абзацы не рвём при накоплении.
        Если абзац сам длиннее лимита — делим его по строкам, строки рвём только если они длиннее лимита (и если строка — одно длинное слово, делим по max_chars)."""
        import re
        text = re.sub(r'\n\s*\n', '\n\n', text)
        paragraphs = text.split("\n\n")
        chunks = []
        buf = []
        buf_len = 0

        def split_paragraph(para: str, max_chars: int, max_word_len: int = 40) -> list[str]:
            lines = para.splitlines()
            chunks = []
            current_chunk = []
            current_len = 0

            for line in lines:
                line = line.strip()
                if not line:
                    continue

                words = line.split()
                # если строка — одно длинное слово (нет пробелов)
                if len(words) == 1 and len(words[0]) > max_chars:
                    if current_chunk:
                        chunks.append("\n".join(current_chunk))
                        current_chunk = []
                        current_len = 0
                    # разбиваем длинное слово на куски по max_word_len
                    for i in range(0, len(line), max_word_len):
                        part = line[i:i+max_word_len]
                        chunks.append(part)
                    continue

                if len(line) > max_chars:
                    if current_chunk:
                        chunks.append("\n".join(current_chunk))
                        current_chunk = []
                        current_len = 0
                    buf = []
                    buf_len = 0
                    for w in words:
                        if buf_len + len(w) + 1 > max_chars:
                            chunks.append(" ".join(buf))
                            buf = [w]
                            buf_len = len(w)
                        else:
                            buf.append(w)
                            buf_len += len(w) + 1
                    if buf:
                        chunks.append(" ".join(buf))
                else:
                    if current_len + len(line) + 1 > max_chars:
                        chunks.append("\n".join(current_chunk))
                        current_chunk = [line]
                        current_len = len(line)
                    else:
                        current_chunk.append(line)
                        current_len += len(line) + 1

            if current_chunk:
                chunks.append("\n".join(current_chunk))
            return [chunk for chunk in chunks if chunk.strip()]

        for para in paragraphs:
            para = para.strip()
            if not para:
                continue

            if len(para) > max_chars:
                if buf:
                    chunks.append("\n\n".join(buf))
                    buf, buf_len = [], 0
                chunks.extend(split_paragraph(para, max_chars, max_word_len=0))
                continue

            if buf_len == 0:
                buf.append(para)
                buf_len = len(para) + 2
            elif buf_len + len(para) + 2 <= max_chars:
                buf.append(para)
                buf_len += len(para) + 2
            else:
                chunks.append("\n\n".join(buf))
                buf = [para]
                buf_len = len(para) + 2

        if buf:
            chunks.append("\n\n".join(buf))

        return chunks

    # --- основная логика safe_process ---
    result = []
    for chunk in chunk_text_by_lines(text, max_chars=max_len):
        print("\n--- ЧАНК ---\n", chunk, "\n")  # <-- вывод чанка
        processed = accentizer.process_all(chunk)
        result.append(processed)

    return "\n\n".join(result)


=======================

!!!!!  Патчнуть класс.      ОТКЛЮЧИТЬ ЁФИКАЦИЮ   !!!
У RUAccent внутри вызывается метод _process_yo(). Можно просто переопределить его. 

In [ ]:
from ruaccent import RUAccent

class RUAccentNoYo(RUAccent):
    def _process_yo(self, words, sentence):
        return words  # ничего не делаем

accentizer = RUAccentNoYo()
accentizer.load(omograph_model_size='big_poetry',
                use_dictionary=True, tiny_mode=False, device="CPU")

text = "когда нибудь ты все увидишь"

print(accentizer.process_all(text))


!!!  Чуть сложнее чем ^  (подменяет оставляя имя RUAccent)
(ещё сложнее (с контекст-отключением и с monkeypatch)  см 1_эксперим\my_ruaccent.py)

In [ ]:
from ruaccent import RUAccent as _RUAccent

class RUAccent(_RUAccent):
    def __init__(self, *args, disable_yo: bool = False, **kwargs):
        super().__init__(*args, **kwargs)
        self._disable_yo = bool(disable_yo)
 
    def disable_yo(self, state: bool = True):
        """Включить/выключить ё-фиксацию на лету."""
        self._disable_yo = bool(state)

    def _process_yo(self, words, sentence):
        # если выключено — не делаем замен; иначе передаём реализацию родителю
        if self._disable_yo:
            return words
        return super()._process_yo(words, sentence)
    
""" далее в др. файле
from my_ruaccent import RUAccent

acc = RUAccent()
acc.load()

text = "Все хорошо`"

print(acc.process_all(text))        # с ёфикацией (по умолчанию)
acc.disable_yo(True)                #     отключаем
print(acc.process_all(text))        # без ёфикации
acc.disable_yo(False)               #     включаем обратно
print(acc.process_all(text))        # снова с ёфикацией

with acc.no_yo():
    print(acc.process_all(text))  # временно без ёфикации
print(acc.process_all(text))   # снова с ёфикацией


acc2 = RUAccent(disable_yo=True)  # сразу экземпляр с отключённой ёфикацией 
acc2.load()
print(acc2.process_all("Все хорошо")) # никаких новых 'ё'

"""

=====================================